In [4]:
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
import numpy as np
import matplotlib.patches as mpatches

In [5]:
df = pd.read_csv("src/data/FoodBalanceSheetsHistoric_E_All_Data.csv", encoding='latin-1')
df.head()

,Area Code,Area,Item Code,Item,Element Code,Element,Unit,Y1961,Y1961F,Y1962,...,Y2009,Y2009F,Y2010,Y2010F,Y2011,Y2011F,Y2012,Y2012F,Y2013,Y2013F
0,2,Afghanistan,2501,Population,511,Total Population - Both sexes,1000 persons,8954.00,NaN,9142.00,...,27708.00,NaN,28398.00,NaN,29105.00,NaN,29825.00,NaN,30552.00,NaN
1,2,Afghanistan,2901,Grand Total,664,Food supply (kcal/capita/day),kcal/capita/day,2999.00,Fc,2917.00,...,2081.00,Fc,2104.00,Fc,2107.00,Fc,2100.00,Fc,2090.00,Fc
2,2,Afghanistan,2901,Grand Total,674,Protein supply quantity (g/capita/day),g/capita/day,84.91,Fc,82.98,...,57.79,Fc,58.14,Fc,58.91,Fc,58.91,Fc,58.25,Fc
3,2,Afghanistan,2901,Grand Total,684,Fat supply quantity (g/capita/day),g/capita/day,37.51,Fc,37.61,...,30.72,Fc,33.88,Fc,33.08,Fc,33.37,Fc,33.52,Fc
4,2,Afghanistan,2903,Vegetal Products,664,Food supply (kcal/capita/day),kcal/capita/day,2752.00,Fc,2672.00,...,1871.00,Fc,1888.00,Fc,1891.00,Fc,1883.00,Fc,1873.00,Fc


In [25]:
mozambique = df[df.Area == 'Mozambique']

grouped_mo = mozambique.groupby('Element').sum()
grouped_mo.loc['Combined'] = grouped_mo.loc['Feed']+grouped_mo.loc['Food']
grouped_mo.loc['Prop_Food'] = grouped_mo.loc['Food']/grouped_mo.loc['Combined']
grouped_mo.loc['Prop_Feed'] = grouped_mo.loc['Feed']/grouped_mo.loc['Combined']
grouped_mo=grouped_mo.drop(['Area Code', 'Item Code', 'Element Code'], axis = 1).reset_index()
grouped_mo = grouped_mo.iloc[[5,17,18]]
grouped_mo

# melted_mo = pd.melt(grouped_mo.iloc[18:19], id_vars=['Element'], value_vars=grouped_mo.columns)
# melted_mo['variable'] = melted_mo.variable.str.lstrip('Y')

,Element,Y1961,Y1962,Y1963,Y1964,Y1965,Y1966,Y1967,Y1968,Y1969,...,Y2004,Y2005,Y2006,Y2007,Y2008,Y2009,Y2010,Y2011,Y2012,Y2013
5,Food supply (kcal/capita/day),7335.000000,7445.000000,7224.000000,7406.000000,7463.000000,7375.000000,7392.00000,7072.000000,7352.000000,...,8232.000000,8146.000000,8439.000000,8333.000000,8649.000000,8656.000000,8965.000000,9033.000000,9191.00000,9129.000000
17,Prop_Food,0.925628,0.924168,0.927009,0.927425,0.928745,0.928244,0.92743,0.928098,0.928862,...,0.915477,0.928065,0.926553,0.925286,0.917592,0.909995,0.882103,0.879224,0.88427,0.879379
18,Prop_Feed,0.074372,0.075832,0.072991,0.072575,0.071255,0.071756,0.07257,0.071902,0.071138,...,0.084523,0.071935,0.073447,0.074714,0.082408,0.090005,0.117897,0.120776,0.11573,0.120621


In [26]:
import json

f = open("src/data/world3.json")
geojson = json.load(f)


In [27]:
area_replacements = [
  ("United Kingdom of Great Britain and Northern Ireland", "United Kingdom"), 
  ("United States of America", "United States"),
  ("Czechia", "Czech Republic"),
  ("Republic of Moldova", "Moldova"),
  ("North Macedonia", "Macedonia"),
  ("Iran (Islamic Republic of)", "Iran"),
  ("Congo", "Democratic Republic of Congo"),
  ("Venezuela (Bolivarian Republic of)", "Venezuela"),
  ("Bolivia (Plurinational State of)", "Bolivia"),
  ("Viet Nam", "Vietnam")
]

for pair in area_replacements:
  df.Area = df.Area.replace(pair[0], pair[1])

In [28]:
a = df[(df.Item == "Grand Total") & (df.Element == "Food supply (kcal/capita/day)")]

for feature in geojson['features']:
  try: 
    area = feature['properties']['NAME_LONG']
    mozambique = df[df.Area == area]

    grouped_mo = mozambique.groupby('Element').sum()
    grouped_mo.loc['Combined'] = grouped_mo.loc['Feed']+grouped_mo.loc['Food']
    grouped_mo.loc['Prop_Food'] = grouped_mo.loc['Food']/grouped_mo.loc['Total Population - Both sexes']
    grouped_mo.loc['Prop_Feed'] = grouped_mo.loc['Feed']/grouped_mo.loc['Total Population - Both sexes']
    grouped_mo=grouped_mo.drop(['Area Code', 'Item Code', 'Element Code'], axis = 1).reset_index()

    ids = [3, 4, 17, 18]
    labels = ["feed", "food", "prop_food", "prop_feed"]

    for idx, label in enumerate(labels):
      feature['properties'][label] = list(grouped_mo.iloc[idx][1:])

    feature['properties']['food_supply'] = list(a[a.Area == area].groupby("Element").sum().values[0][3:])
  except KeyError:
    print("Country not found: ", area)
  
  

Country not found:  Aland Islands
Country not found:  Andorra
Country not found:  French Southern and Antarctic Lands
Country not found:  Burundi
Country not found:  Bahrain
Country not found:  Bhutan
Country not found:  Cyprus U.N. Buffer Zone
Country not found:  Democratic Republic of the Congo
Country not found:  Republic of Congo
Country not found:  Comoros
Country not found:  Cape Verde
Country not found:  Northern Cyprus
Country not found:  Eritrea
Country not found:  Dhekelia
Country not found:  Falkland Islands
Country not found:  Faeroe Islands
Country not found:  The Gambia
Country not found:  Equatorial Guinea
Country not found:  Greenland
Country not found:  Hong Kong
Country not found:  Isle of Man
Country not found:  Baikonur Cosmodrome
Country not found:  Siachen Glacier
Country not found:  Kosovo
Country not found:  Lao PDR
Country not found:  Libya
Country not found:  Papua New Guinea
Country not found:  Puerto Rico
Country not found:  Dem. Rep. Korea
Country not found

In [30]:
out = open("src/data/out.json", "w+")
json.dump(geojson, out)

In [23]:

mozambique = df[df.Area == "Mozambique"]

grouped_mo = mozambique.groupby('Element').sum()
grouped_mo.loc['Combined'] = grouped_mo.loc['Feed']+grouped_mo.loc['Food']
grouped_mo.loc['Prop_Food'] = grouped_mo.loc['Food']/grouped_mo.loc['Total Population - Both sexes']
grouped_mo.loc['Prop_Feed'] = grouped_mo.loc['Feed']/grouped_mo.loc['Total Population - Both sexes']
grouped_mo=grouped_mo.drop(['Area Code', 'Item Code', 'Element Code'], axis = 1).reset_index()
# grouped_mo = grouped_mo.iloc[[3, 45,17,18]]
grouped_mo
# list(grouped_mo.iloc[3])

,Element,Y1961,Y1962,Y1963,Y1964,Y1965,Y1966,Y1967,Y1968,Y1969,...,Y2004,Y2005,Y2006,Y2007,Y2008,Y2009,Y2010,Y2011,Y2012,Y2013
0,Domestic supply quantity,11495.260000,12080.860000,11711.090000,11729.600000,11975.280000,12229.420000,13332.980000,14034.260000,15009.380000,...,26895.820000,26369.950000,27437.300000,27321.550000,29735.280000,31600.430000,37058.660000,39650.300000,43212.300000,44048.300000
1,Export Quantity,1292.300000,1349.960000,1293.200000,1190.800000,990.300000,1118.930000,1184.140000,1658.260000,1438.700000,...,306.900000,367.920000,714.190000,592.860000,585.000000,546.620000,999.910000,933.490000,1016.490000,838.490000
2,Fat supply quantity (g/capita/day),83.240000,87.720000,80.820000,93.910000,93.670000,98.730000,103.280000,101.750000,109.380000,...,144.050000,155.680000,147.960000,146.660000,156.980000,146.270000,157.620000,162.330000,167.500000,167.640000
3,Feed,577.000000,609.000000,577.000000,594.000000,594.000000,603.000000,627.000000,616.000000,637.000000,...,1675.920000,1361.640000,1489.220000,1522.980000,1821.460000,2101.430000,3028.620000,3209.780000,3253.780000,3439.780000
4,Food,7181.260000,7421.860000,7328.090000,7590.600000,7742.280000,7800.420000,8012.980000,7951.260000,8317.380000,...,18152.060000,17567.190000,18786.800000,18861.190000,20281.410000,21246.520000,22660.050000,23366.530000,24861.530000,25077.530000
5,Food supply (kcal/capita/day),7335.000000,7445.000000,7224.000000,7406.000000,7463.000000,7375.000000,7392.000000,7072.000000,7352.000000,...,8232.000000,8146.000000,8439.000000,8333.000000,8649.000000,8656.000000,8965.000000,9033.000000,9191.000000,9129.000000
6,Food supply quantity (kg/capita/yr),920.650000,932.650000,902.560000,914.780000,914.250000,901.710000,906.330000,879.460000,899.700000,...,888.210000,836.250000,870.200000,850.900000,891.100000,909.440000,945.530000,950.610000,986.560000,970.780000
7,Import Quantity,251.960000,356.210000,375.290000,348.600000,450.180000,440.760000,455.120000,427.120000,519.080000,...,2602.810000,2777.310000,3211.320000,2366.800000,2390.900000,2953.130000,2919.490000,2777.090000,3069.090000,3141.090000
8,Losses,385.000000,391.000000,392.000000,392.000000,396.000000,399.000000,433.000000,454.000000,468.000000,...,2020.000000,1360.000000,1668.000000,1423.000000,1282.000000,1669.000000,2255.000000,2352.000000,2315.000000,2327.000000
9,Other uses (non-food),12.000000,14.000000,14.000000,14.000000,14.000000,16.000000,16.000000,16.000000,16.000000,...,151.840000,135.120000,152.300000,133.380000,214.400000,879.480000,2446.000000,2672.000000,4705.000000,4407.000000


In [21]:
grouped = df.groupby('Element').sum()
grouped.loc['Combined'] = grouped.loc['Feed']+grouped.loc['Food']
grouped.loc['Prop_Food'] = grouped.loc['Food']/grouped.loc['Combined']
grouped.loc['Prop_Feed'] = grouped.loc['Feed']/grouped.loc['Combined']
grouped=grouped.drop(['Area Code', 'Item Code', 'Element Code'], axis = 1).reset_index()
grouped.iloc[4].mean

<bound method NDFrame._add_numeric_operations.<locals>.mean of Element           Food
Y1961      14167404.24
Y1962      14553648.68
Y1963      14800692.71
Y1964      15151075.18
Y1965      15648214.11
Y1966      16137188.34
Y1967      16518483.22
Y1968       16926715.6
Y1969      17390735.63
Y1970      17910556.04
Y1971      18124543.13
Y1972      18266472.68
Y1973      19023930.27
Y1974      19389058.76
Y1975      19818062.48
Y1976       20062780.1
Y1977      20626304.94
Y1978      21380490.43
Y1979      21733056.35
Y1980      21989721.49
Y1981      22431743.63
Y1982      23070481.05
Y1983       23722190.3
Y1984      24342005.66
Y1985      24795958.19
Y1986      25406894.64
Y1987      26069954.16
Y1988      26600181.56
Y1989      27175995.69
Y1990      27562072.73
Y1991      27852822.28
Y1992      28769965.54
Y1993      29516838.37
Y1994      30324685.08
Y1995      31346712.09
Y1996      32132718.01
Y1997      32950610.58
Y1998      33860697.77
Y1999      35060176.27
Y2000       36028